In [1]:
#This loads required libraries. 
import importlib
import ruamel.yaml
import pymongo
from pymongo import MongoClient#A .yaml file
cf_file="vm.yaml"
with open(cf_file, 'r') as yaml:
    cf=ruamel.yaml.round_trip_load(yaml, preserve_quotes=True)
client = MongoClient(cf['mongo_uri'])
db = client[cf['mongo_db']]

#process = cp.CrunchBaseProcessor('mac.yaml')
#total=process.count_mongo()

In [2]:
# import pandas as pd
# #This obtains all documents in a collection.  In the "find" is where you have a specific query. 
# collection = db.acquisitions
# #Tutorial
# #https://docs.mongodb.com/manual/tutorial/project-fields-from-query-results/
# cursor = collection.find({ }, { 'data.relationships.acquiree.item.uuid': 1, 'data.relationships.acquiree.item.properties.permalink': 1,  
#                                 'data.relationships.acquirer.item.uuid': 1, 'data.relationships.acquirer.item.properties.permalink': 1, 
#                                 '_id':0} )

# 1. Data Properties

In [3]:
import pandas as pd
#This obtains all documents in a collection.  In the "find" is where you have a specific query. 
collection = db.acquisitions
#Tutorial
#https://docs.mongodb.com/manual/tutorial/project-fields-from-query-results/
cursor = collection.find({ }, {  'data.properties.price_currency_code': 1,'data.properties.price_usd': 1,
                               'data.properties.payment_type': 1,'data.properties.acquisition_type': 1,
                               'data.properties.acquisition_status': 1,'data.properties.disposition_of_acquired': 1,
                               'data.properties.announced_on': 1,'data.properties.announced_on_trust_code': 1,
                               'data.properties.completed_on': 1,'data.properties.completed_on_trust_code': 1,
                               'data.properties.rank': 1,'data.properties.created_at': 1,'data.properties.updated_at': 1,
                               '_id':0} )

In [4]:
cursor_proper = pd.DataFrame(list(cursor))
cursor_proper.head()

,data
0,"{'properties': {'price_currency_code': 'USD', ..."
1,"{'properties': {'price_currency_code': 'USD', ..."
2,"{'properties': {'price_currency_code': None, '..."
3,"{'properties': {'price_currency_code': 'USD', ..."
4,"{'properties': {'price_currency_code': None, '..."


In [5]:
cursor_proper['price_currency_code'] = cursor_proper['data'].apply(lambda x: x['properties']['price_currency_code'])
cursor_proper['price_usd'] = cursor_proper['data'].apply(lambda x: x['properties']['price_usd'])
cursor_proper['payment_type'] = cursor_proper['data'].apply(lambda x: x['properties']['payment_type'])
cursor_proper['acquisition_type'] = cursor_proper['data'].apply(lambda x: x['properties']['acquisition_type'])
cursor_proper['acquisition_status'] = cursor_proper['data'].apply(lambda x: x['properties']['acquisition_status'])
cursor_proper['disposition_of_acquired'] = cursor_proper['data'].apply(lambda x: x['properties']['disposition_of_acquired'])
cursor_proper['announced_on'] = cursor_proper['data'].apply(lambda x: x['properties']['announced_on'])
cursor_proper['announced_on_trust_code'] = cursor_proper['data'].apply(lambda x: x['properties']['announced_on_trust_code'])
cursor_proper['completed_on'] = cursor_proper['data'].apply(lambda x: x['properties']['completed_on'])
cursor_proper['completed_on_trust_code'] = cursor_proper['data'].apply(lambda x: x['properties']['completed_on_trust_code'])
cursor_proper['rank'] = cursor_proper['data'].apply(lambda x: x['properties']['rank'])
cursor_proper['created_at'] = cursor_proper['data'].apply(lambda x: x['properties']['created_at'])
cursor_proper['updated_at'] = cursor_proper['data'].apply(lambda x: x['properties']['updated_at'])
cursor_proper.head()

,data,price_currency_code,price_usd,payment_type,acquisition_type,acquisition_status,disposition_of_acquired,announced_on,announced_on_trust_code,completed_on,completed_on_trust_code,rank,created_at,updated_at
0,"{'properties': {'price_currency_code': 'USD', ...",USD,1.050000e+10,None,acquisition,complete,None,2013-08-24,7,None,NaN,35434,1462342074,1518479436
1,"{'properties': {'price_currency_code': 'USD', ...",USD,4.800000e+06,None,acquisition,complete,separate_entity,2014-12-25,7,None,NaN,35409,1419567767,1518478268
2,"{'properties': {'price_currency_code': None, '...",None,NaN,None,acquisition,complete,None,2000-02-14,7,None,NaN,63090,1452501057,1518479611
3,"{'properties': {'price_currency_code': 'USD', ...",USD,2.982200e+08,None,acquisition,pending,division,2017-10-09,7,None,NaN,11784,1507548617,1518478487
4,"{'properties': {'price_currency_code': None, '...",None,NaN,None,acquisition,complete,subsidiary,2000-04-03,7,None,NaN,17987,1504180959,1518479519


In [6]:
len(cursor_proper)

72295

In [7]:
cursor_proper.isnull().sum()

data                           0
price_currency_code        57992
price_usd                  57992
payment_type               63087
acquisition_type            8644
acquisition_status         10006
disposition_of_acquired    33434
announced_on                   0
announced_on_trust_code        0
completed_on               66912
completed_on_trust_code    66912
rank                           0
created_at                     0
updated_at                     0
dtype: int64

# 2. Acquiree and Acquirer variables

In [11]:
import pandas as pd
#This obtains all documents in a collection.  In the "find" is where you have a specific query. 
collection = db.acquisitions
#Tutorial
#https://docs.mongodb.com/manual/tutorial/project-fields-from-query-results/
cursor2 = collection.find({ }, {  
                               'data.relationships.acquiree.item.properties.permalink_aliases': 1, 'data.relationships.acquiree.item.properties.api_path': 1,  
                               'data.relationships.acquiree.item.properties.web_path': 1, 'data.relationships.acquiree.item.properties.api_url': 1, 
                               'data.relationships.acquiree.item.properties.name': 1, 'data.relationships.acquiree.item.properties.also_known_as': 1, 
                               'data.relationships.acquiree.item.properties.short_description': 1, 'data.relationships.acquiree.item.properties.description': 1, 
                               'data.relationships.acquiree.item.properties.profile_image_url': 1, 'data.relationships.acquiree.item.properties.primary_role': 1, 
                               'data.relationships.acquiree.item.properties.role_company': 1, 'data.relationships.acquiree.item.properties.role_investor': 1, 
                               'data.relationships.acquiree.item.properties.role_group': 1, 'data.relationships.acquiree.item.properties.role_school': 1, 
                               'data.relationships.acquiree.item.properties.investor_type': 1, 'data.relationships.acquiree.item.properties.founded_on': 1, 
                               'data.relationships.acquiree.item.properties.founded_on_trust_code': 1, 'data.relationships.acquiree.item.properties.is_closed': 1,
                               'data.relationships.acquiree.item.properties.closed_on': 1, 'data.relationships.acquiree.item.properties.closed_on_trust_code': 1,  
                               'data.relationships.acquiree.item.properties.num_employees_min': 1, 'data.relationships.acquiree.item.properties.num_employees_max': 1, 
                               'data.relationships.acquiree.item.properties.stock_exchange': 1, 'data.relationships.acquiree.item.properties.stock_symbol': 1, 
                               'data.relationships.acquiree.item.properties.total_funding_usd': 1, 'data.relationships.acquiree.item.properties.number_of_investments': 1, 
                               'data.relationships.acquiree.item.properties.homepage_url': 1, 'data.relationships.acquiree.item.properties.contact_email': 1, 
                               'data.relationships.acquiree.item.properties.phone_number': 1, 'data.relationships.acquiree.item.properties.rank': 1, 
                               'data.relationships.acquiree.item.properties.created_at': 1, 'data.relationships.acquiree.item.properties.updated_at': 1, 
                               'data.relationships.acquirer.item.properties.permalink_aliases': 1, 'data.relationships.acquirer.item.properties.api_path': 1,  
                               'data.relationships.acquirer.item.properties.web_path': 1, 'data.relationships.acquirer.item.properties.api_url': 1, 
                               'data.relationships.acquirer.item.properties.name': 1, 'data.relationships.acquirer.item.properties.also_known_as': 1, 
                               'data.relationships.acquirer.item.properties.short_description': 1, 'data.relationships.acquirer.item.properties.description': 1, 
                               'data.relationships.acquirer.item.properties.profile_image_url': 1, 'data.relationships.acquirer.item.properties.primary_role': 1, 
                               'data.relationships.acquirer.item.properties.role_company': 1, 'data.relationships.acquirer.item.properties.role_investor': 1, 
                               'data.relationships.acquirer.item.properties.role_group': 1, 'data.relationships.acquirer.item.properties.role_school': 1, 
                               'data.relationships.acquirer.item.properties.investor_type': 1, 'data.relationships.acquirer.item.properties.founded_on': 1, 
                               'data.relationships.acquirer.item.properties.founded_on_trust_code': 1, 'data.relationships.acquirer.item.properties.is_closed': 1,
                               'data.relationships.acquirer.item.properties.closed_on': 1, 'data.relationships.acquirer.item.properties.closed_on_trust_code': 1,  
                               'data.relationships.acquirer.item.properties.num_employees_min': 1, 'data.relationships.acquirer.item.properties.num_employees_max': 1, 
                               'data.relationships.acquirer.item.properties.stock_exchange': 1, 'data.relationships.acquirer.item.properties.stock_symbol': 1, 
                               'data.relationships.acquirer.item.properties.total_funding_usd': 1, 'data.relationships.acquirer.item.properties.number_of_investments': 1, 
                               'data.relationships.acquirer.item.properties.homepage_url': 1, 'data.relationships.acquirer.item.properties.contact_email': 1, 
                               'data.relationships.acquirer.item.properties.phone_number': 1, 'data.relationships.acquirer.item.properties.rank': 1, 
                               'data.relationships.acquirer.item.properties.created_at': 1, 'data.relationships.acquirer.item.properties.updated_at': 1,                             
                               '_id':0} )

In [12]:
cursor_acqu = pd.DataFrame(list(cursor2))
cursor_acqu.head()

,data
0,{'relationships': {'acquiree': {'item': {'prop...
1,{'relationships': {'acquiree': {'item': {'prop...
2,{'relationships': {'acquiree': {'item': {'prop...
3,{'relationships': {'acquiree': {'item': {'prop...
4,{'relationships': {'acquiree': {'item': {'prop...


In [14]:
cursor_acqu['acquiree_permalink_aliases'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['permalink_aliases'])
cursor_acqu['acquiree_api_path'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['api_path'])
cursor_acqu['acquiree_web_path'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['web_path'])
cursor_acqu['acquiree_api_url'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['api_url'])
cursor_acqu['acquiree_name'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['name'])
cursor_acqu['acquiree_also_known_as'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['also_known_as'])
cursor_acqu['acquiree_short_description'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['short_description'])
cursor_acqu['acquiree_description'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['description'])
cursor_acqu['acquiree_profile_image_url'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['profile_image_url'])
cursor_acqu['acquiree_primary_role'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['primary_role'])
cursor_acqu['acquiree_role_company'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['role_company'])
cursor_acqu['acquiree_role_investor'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['role_investor'])
cursor_acqu['acquiree_role_group'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['role_group'])
cursor_acqu['acquiree_role_school'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['role_school'])
cursor_acqu['acquiree_investor_type'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['investor_type'])
cursor_acqu['acquiree_founded_on'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['founded_on'])
cursor_acqu['acquiree_founded_on_trust_code'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['founded_on_trust_code'])
cursor_acqu['acquiree_is_closed'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['is_closed'])
cursor_acqu['acquiree_closed_on'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['closed_on'])
cursor_acqu['acquiree_closed_on_trust_code'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['closed_on_trust_code'])
cursor_acqu['acquiree_num_employees_min'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['num_employees_min'])
cursor_acqu['acquiree_num_employees_max'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['num_employees_max'])
cursor_acqu['acquiree_stock_exchange'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['stock_exchange'])
cursor_acqu['acquiree_stock_symbol'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['stock_symbol'])
cursor_acqu['acquiree_total_funding_usd'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['total_funding_usd'])
cursor_acqu['acquiree_number_of_investments'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['number_of_investments'])
cursor_acqu['acquiree_homepage_url'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['homepage_url'])
cursor_acqu['acquiree_contact_email'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['contact_email'])
cursor_acqu['acquiree_phone_number'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['phone_number'])
cursor_acqu['acquiree_rank'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['rank'])
cursor_acqu['acquiree_created_at'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['created_at'])
cursor_acqu['acquiree_updated_at'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['updated_at'])

cursor_acqu['acquirer_permalink_aliases'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['permalink_aliases'])
cursor_acqu['acquirer_api_path'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['api_path'])
cursor_acqu['acquirer_web_path'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['web_path'])
cursor_acqu['acquirer_api_url'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['api_url'])
cursor_acqu['acquirer_name'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['name'])
cursor_acqu['acquirer_also_known_as'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['also_known_as'])
cursor_acqu['acquirer_short_description'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['short_description'])
cursor_acqu['acquirer_description'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['description'])
cursor_acqu['acquirer_profile_image_url'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['profile_image_url'])
cursor_acqu['acquirer_primary_role'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['primary_role'])
cursor_acqu['acquirer_role_company'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['role_company'])
cursor_acqu['acquirer_role_investor'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['role_investor'])
cursor_acqu['acquirer_role_group'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['role_group'])
cursor_acqu['acquirer_role_school'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['role_school'])
cursor_acqu['acquirer_investor_type'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['investor_type'])
cursor_acqu['acquirer_founded_on'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['founded_on'])
cursor_acqu['acquirer_founded_on_trust_code'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['founded_on_trust_code'])
cursor_acqu['acquirer_is_closed'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['is_closed'])
cursor_acqu['acquirer_closed_on'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['closed_on'])
cursor_acqu['acquirer_closed_on_trust_code'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['closed_on_trust_code'])
cursor_acqu['acquirer_num_employees_min'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['num_employees_min'])
cursor_acqu['acquirer_num_employees_max'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['num_employees_max'])
cursor_acqu['acquirer_stock_exchange'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['stock_exchange'])
cursor_acqu['acquirer_stock_symbol'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['stock_symbol'])
cursor_acqu['acquirer_total_funding_usd'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['total_funding_usd'])
cursor_acqu['acquirer_number_of_investments'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['number_of_investments'])
cursor_acqu['acquirer_homepage_url'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['homepage_url'])
cursor_acqu['acquirer_contact_email'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['contact_email'])
cursor_acqu['acquirer_phone_number'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['phone_number'])
cursor_acqu['acquirer_rank'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['rank'])
cursor_acqu['acquirer_created_at'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['created_at'])
cursor_acqu['acquirer_updated_at'] = cursor_acqu['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['updated_at'])
cursor_acqu.head()

,data,acquiree_permalink_aliases,acquiree_api_path,acquiree_web_path,acquiree_api_url,acquiree_name,acquiree_also_known_as,acquiree_short_description,acquiree_description,acquiree_profile_image_url,...,acquirer_stock_exchange,acquirer_stock_symbol,acquirer_total_funding_usd,acquirer_number_of_investments,acquirer_homepage_url,acquirer_contact_email,acquirer_phone_number,acquirer_rank,acquirer_created_at,acquirer_updated_at
0,{'relationships': {'acquiree': {'item': {'prop...,[],organizations/onyx-pharmaceuticals,organization/onyx-pharmaceuticals,https://api.crunchbase.com/v3.1/organizations/...,Onyx Pharmaceuticals,None,Onyx Pharmaceuticals is a biopharmaceutical co...,Onyx Pharmaceuticals is a biopharmaceutical co...,http://public.crunchbase.com/t_api_images/v139...,...,nasdaq,AMGN,0,6,None,amgenmediarelations@amgen.com,(805)447-1000,21280,1462277987,1523953054
1,{'relationships': {'acquiree': {'item': {'prop...,[],organizations/’compass-solutions,organization/’compass-solutions,https://api.crunchbase.com/v3.1/organizations/...,N’compass Solutions,None,N’compass Solutions is a software solutions an...,"compass Solutions, Inc. is a Minneapolis-based...",http://public.crunchbase.com/t_api_images/v141...,...,otcqb,APPO,0,0,None,info@alphapoint-us.com,(941) 907-8822,117256,1241171903,1518482277
2,{'relationships': {'acquiree': {'item': {'prop...,[],organizations/comfort-living,organization/comfort-living,https://api.crunchbase.com/v3.1/organizations/...,Comfort Living,None,"An online retailer of personal care, home heal...",None,None,...,None,None,0,0,None,support@more.com.,None,97799,1386418506,1523958401
3,{'relationships': {'acquiree': {'item': {'prop...,None,organizations/afw-uk-oil-gas,organization/afw-uk-oil-gas,https://api.crunchbase.com/v3.1/organizations/...,AFW UK Oil & Gas,None,AFW UK Oil & Gas provides engineering and cons...,"AFW UK provides a range of services, including...",None,...,None,None,0,0,None,None,None,80656,1279883947,1523957504
4,{'relationships': {'acquiree': {'item': {'prop...,[],organizations/jt-usa-inc,organization/jt-usa-inc,https://api.crunchbase.com/v3.1/organizations/...,"JT USA, Inc",None,"JT USA, Inc is a manufacturer of protective ac...","JT USA, Inc is a manufacturer of protective ac...",None,...,nasdaq,XTRM,0,0,None,None,None,379339,1407237757,1518478798


In [15]:
len(cursor_acqu)

72295

In [16]:
cursor_acqu.isnull().sum()

data                                  0
acquiree_permalink_aliases        13525
acquiree_api_path                     0
acquiree_web_path                     0
acquiree_api_url                      0
acquiree_name                         0
acquiree_also_known_as            65392
acquiree_short_description            0
acquiree_description               5747
acquiree_profile_image_url        20199
acquiree_primary_role                 0
acquiree_role_company                 0
acquiree_role_investor                0
acquiree_role_group                   0
acquiree_role_school                  0
acquiree_investor_type            72055
acquiree_founded_on               12003
acquiree_founded_on_trust_code    12003
acquiree_is_closed                    0
acquiree_closed_on                71815
acquiree_closed_on_trust_code     71343
acquiree_num_employees_min        32646
acquiree_num_employees_max        32646
acquiree_stock_exchange           69999
acquiree_stock_symbol             69997


In [6]:
# cursor_df['acquiree_uuid'] = cursor_df['data'].apply(lambda x: x['relationships']['acquiree']['item']['uuid'])
# cursor_df['acquiree_permalink'] = cursor_df['data'].apply(lambda x: x['relationships']['acquiree']['item']['properties']['permalink'])
# cursor_df['acquirer_uuid'] = cursor_df['data'].apply(lambda x: x['relationships']['acquirer']['item']['uuid'])
# cursor_df['acquirer_permalink'] = cursor_df['data'].apply(lambda x: x['relationships']['acquirer']['item']['properties']['permalink'])
# cursor_df.head()

In [16]:
# cursor_df.drop(['data'],axis=1).to_csv('task1_uuid&permalink.csv')